### Imports

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

### Import Pixar csv

In [8]:
pixar_df = pd.read_csv('../data/pixar__studio12.csv')
pixar_df.sample(5)

,Moviename,Releaseyear,Budget(million),US and canada(million),Other territories(million),Worldwide(million),Rotten Tomatoes
9,Up,2009,1750,2930,4421,7351,98%
10,Toy Story 3,2010,2000,4150,6520,10670,98%
3,"Monsters, Inc.",2001,1150,2899,3424,6323,96%
16,Finding Dory,2016,2000,4863,5423,10286,94%
13,Monsters University,2013,2000,2685,4751,7436,80%


## Scrapping wiki table

In [15]:
wiki = 'https://en.wikipedia.org/wiki/List_of_Pixar_films'

In [14]:
res = requests.get(wiki)
soup = BeautifulSoup(res.content, "html.parser")

In [13]:
wrappers = soup.find_all('table', {'class':'wikitable'})
wrappers[4]

<table class="wikitable sortable" style="text-align:center; margin=auto; font-size:90%">
<tbody><tr>
<th rowspan="2" scope="col">Film
</th>
<th rowspan="2" scope="col"><a href="/wiki/Academy_Award_for_Best_Picture" title="Academy Award for Best Picture">Best Picture</a>
</th>
<th rowspan="2" scope="col"><a href="/wiki/Academy_Award_for_Best_Animated_Feature" title="Academy Award for Best Animated Feature">Animated Feature</a>
</th>
<th rowspan="2" scope="col"><a href="/wiki/Academy_Award_for_Best_Original_Screenplay" title="Academy Award for Best Original Screenplay">Original Screenplay</a>
</th>
<th rowspan="2" scope="col"><a href="/wiki/Academy_Award_for_Best_Adapted_Screenplay" title="Academy Award for Best Adapted Screenplay">Adapted Screenplay</a>
</th>
<th rowspan="2" scope="col"><a href="/wiki/Academy_Award_for_Best_Original_Score" title="Academy Award for Best Original Score">Original Score</a>
</th>
<th rowspan="2" scope="col"><a href="/wiki/Academy_Award_for_Best_Original_Son

### Column names extraction

In [23]:
col_names = wrappers[4].find_all('th')
col_names = [i.getText().strip() for i in col_names]
col_names.remove('Sound[a]')
col_names = [i.lower().replace(' ', '_') for i in col_names]
col_names

['film',
 'best_picture',
 'animated_feature',
 'original_screenplay',
 'adapted_screenplay',
 'original_score',
 'original_song',
 'other',
 'sound_editing',
 'sound_mixing']

### Rows extraction

In [ ]:
def extract_rows_info(rows, col_names):
    data = []
    prev_values = {}  # Para realizar un seguimiento de los valores anteriores de las celdas compartidas
    for row in rows:
        cols = row.find_all('td')
        cols = [col.text.strip() for col in cols]  # Utiliza strip para eliminar espacios en blanco
        for i in range(len(cols)):
            if cols[i] == '':
                # Si la celda está vacía, verifica el valor del atributo rowspan en la celda de la fila anterior
                rowspan = int(cols[i - 1]['rowspan']) if i > 0 else 1
                cols[i] = prev_values.get(col_names[i], '') * rowspan
            else:
                prev_values[col_names[i]] = cols[i]
        data.append(cols)
    return pd.DataFrame(data, columns=col_names)

In [42]:
def extract_rows_info(rows, col_names):
    data = []
    prev_values = {}  # Para realizar un seguimiento de los valores anteriores de las celdas compartidas
    for idx, i in enumerate(rows):
        cols = i.find_all('td')
        # cols = [col.text.strip() for col in cols]  # Utiliza strip para eliminar espacios en blanco
        # for i in range(len(cols)):
        if i == '':
            # Si la celda está vacía, verifica el valor del atributo rowspan en la celda de la fila anterior
            rowspan = int(cols[idx- 1].get('rowspam')) if i > 0 else 1
            cols[idx] = prev_values.get(col_names[idx], '') * rowspan
        else:
            prev_values[col_names[idx]] = cols[idx]
        data.append(cols)
    return pd.DataFrame(data, columns=col_names)


rows = wrappers[4].find_all('tr')
rows = rows[2::]
rows
df = extract_rows_info(rows, col_names)
df

IndexError: list index out of range